In [35]:
import pandas as pd
import sys
sys.path.append("/Users/damoncrockett/ivpy/src")
from ivpy import *
import warnings
warnings.filterwarnings('ignore')

In [36]:
db = pd.read_pickle("/Users/damoncrockett/lmlproc/dbdfgepneural.pkl")

In [37]:
db.rename(columns={"g60":"gloss","ebins":"expressivenessGroup","tbins":"isoGroup"},inplace=True)

In [38]:
for i in db.index:
    catalog = db.catalog.loc[i]
    sb = db.sb.loc[i]
    
    if not sb:
        for j,c in enumerate(catalog):
            if c.isalpha():
                newcatalog = f'{catalog[:j]}_{catalog[j:]}'
                db.catalog.loc[i] = newcatalog
                #print(f'Changed {catalog} to {newcatalog}')

In [39]:
pbcols = [
    'catalog','man','bran','year','circa',
    'surf','xword', 'gword', 'cword', 'tword',
    'backp','postcard','toner','resin','sb','sbid','suffs',
    'dmin', 'dmax', 'dminHex', 'dmaxHex', 'auc', 'roughness','gloss','thickness',
    'expressiveness','radarGroup','expressivenessGroup','isoGroup',
    'gep100','gep150','gep200','gep250','gep300','tmap'
]

In [40]:
def whiten(s):
    try:
        if "specified]" in s or "[Indeterminate]" in s:
            return ''
        else:
            return s
    except:
        return s

In [41]:
db = db[pbcols].applymap(whiten)

In [42]:
db.rename(columns={
    "xword":"textureWord",
    "gword":"glossWord",
    "cword":"colorWord",
    "tword":"thicknessWord"
},inplace=True)

In [43]:
import numpy as np

In [44]:
db.roughness = db.roughness.apply(lambda x: round(x,3))
db.auc = db.auc.apply(lambda x: round(x,3))
db.gloss = db.gloss.apply(lambda x: round(x,3))
db.thickness = db.thickness.apply(lambda x: round(x,3))
db.expressiveness = db.expressiveness.apply(lambda x: round(x,3))

# center GEP

In [45]:
fullrows = db.loc[db.radarGroup.notnull()].index

In [46]:
def makeshift(l,xshift,yshift):
    return [l[0]-xshift,l[1]-yshift,0]

In [47]:
for spread_factor in [100,150,200,250,300]:
    gepcol = f'gep{spread_factor}'
    gepdf = db[gepcol].loc[fullrows]
    
    xrange = gepdf.apply(lambda x: x[0]).max() - gepdf.apply(lambda x: x[0]).min()
    xshift = int(xrange/2)
    
    yrange = gepdf.apply(lambda x: x[1]).max() - gepdf.apply(lambda x: x[1]).min()
    yshift = int(yrange/2)
    
    gepdf = gepdf.apply(lambda l: makeshift(l,xshift,yshift))
    db[gepcol].loc[fullrows] = gepdf

# JSON

In [48]:
def fillrand(cell):
    if isinstance(cell,list):
        return cell
    else:
        return [
            np.random.randint(200,500),
            np.random.randint(200,500),
            0
        ]

In [49]:
db[[c for c in db.columns if 'gep' in c]] = db[[c for c in db.columns if 'gep' in c]].applymap(fillrand)

In [50]:
db.tmap = db.tmap.apply(fillrand)

In [51]:
db = db.fillna('')

#### Has processing instructions

In [52]:
import glob,os

In [53]:
allproc = glob.glob(os.path.join("/Users/damoncrockett/paperbase/src/assets/img/processing/","*.pdf"))

In [54]:
procats = [os.path.basename(item)[:-4] for item in allproc]

In [55]:
db['processing'] = 0

In [56]:
db.processing.loc[db.catalog.isin(procats)] = 1

In [61]:
db[['circa','postcard','resin','sb','processing']] = db[['circa','postcard','resin','sb','processing']].applymap(str)

In [62]:
dbjson = {col: db[col].tolist() for col in db.columns}

In [63]:
import json

with open('/Users/damoncrockett/paperbase/src/assets/data/data.json', 'w') as json_file:
    json.dump(dbjson, json_file)